In [6]:
import requests
from datetime import datetime as dt, timedelta
import pandas as pd

In [7]:
# Definir a URL da API para o endpoint de projetos
url = "https://dadosabertos.camara.leg.br/api/v2/proposicoes"

# Definir os parâmetros da requisição
data_inicio = (datetime.datetime.now() - datetime.timedelta(days=360)).strftime("%Y-%m-%d")
data_fim = datetime.datetime.now().strftime("%Y-%m-%d")
palavras_chave = [ 
"superendividamento",
"inadimplimento das obrigações", 
"mínimo existencial",   
"repactuação de dívidas",
"taxa de juros"
"crédito ao consumidor",
"parcelamento de dívidas",
"renegociação de dívidas"
"rotativo"
"cartão de crédito",
"crédito rural",
"crédito habitacional",
"empréstimo consignado"
"capital de giro",
"crédito para investimento",
"sistemas de informação de crédito",
"ativo problemático",
"crédito a vencer"
]
params = {
    "dataInicio": data_inicio,
    "dataFim": data_fim,
    "ordenarPor": "id",
    "itens": 100,  # Quantidade de itens por página
    "pagina": 1,   # Começar pela primeira página
    "siglaTipo": ["PL", "PLP", "MPV"],
    "keywords": palavras_chave
}
# Definir as palavras-chave que deseja filtrar na ementa dos projetos

# Fazer requisições para todas as páginas de resultados
projetos = []
while True:
    # Fazer a requisição para a API
    response = requests.get(url, params=params)
    # Verificar se a requisição foi bem-sucedida
    if response.status_code == 200:
        # Acessar o conteúdo da resposta em formato JSON
        dados = response.json()["dados"]
        # Verificar se há projetos na página atual
        if len(dados) == 0:
            break
        # Filtrar os projetos que contêm pelo menos uma palavra-chave na ementa
        projetos.extend([projeto for projeto in dados if any(palavra in projeto["ementa"].lower() for palavra in palavras_chave)])
        # Avançar para a próxima página
        params["pagina"] += 1
    else:
        print("Erro ao fazer requisição para a API:", response.status_code)
        break

token = "seu_token_de_acesso_aqui"

# Definir o período de tempo desejado (última semana)
data_atual = dt.now()
data_inicio = data_atual - timedelta(days=7)

# Percorrer a lista de proposições
for proposicao in projetos:
    id_proposicao = proposicao['id']
    
    # Fazer uma chamada ao endpoint /proposicoes/{id}/tramitacoes para obter as tramitações da proposição
    url_tramitacoes = f"https://dadosabertos.camara.leg.br/api/v2/proposicoes/{id_proposicao}/tramitacoes"
    response_tramitacoes = requests.get(url_tramitacoes, headers={"Authorization": f"Bearer {token}"})
    
    if response_tramitacoes.status_code == 200:
        tramitacoes = response_tramitacoes.json()['dados']
        
        # Obter a última tramitação da proposição
        ultima_tramitacao = tramitacoes[-1]
        
        # Extrair a situação de tramitação dessa última tramitação
        situacao_tramitacao = ultima_tramitacao['descricaoSituacao']
        
        # Adicionar a situação de tramitação à proposição
        proposicao['situacaoTramitacao'] = situacao_tramitacao
    else:
        print(f"Erro ao obter as tramitações da proposição {id_proposicao}: {response_tramitacoes.status_code}")

colunas = ['siglaTipo', 'numero', 'ano', 'ementa', 'situacaoTramitacao']

df = pd.DataFrame(projetos, columns=colunas)
df['situacaoTramitacao'] = df['situacaoTramitacao'].astype('str')
df['situacaoTramitacao']=df['situacaoTramitacao'].replace(to_replace='None', value='Não informado')
df.columns = ["Tipo", "Número", "Ano", "Ementa", "Situação"]
df.head(5)

,Tipo,Número,Ano,Ementa,Situação
0,PL,2911,1992,"Altera disposições do Decreto-lei nº 167, de 1...",Arquivada
1,PL,3155,1992,Dispõe sobre títulos de crédito rural e dá out...,Arquivada
2,PL,7645,2006,"Introduz alterações na Lei nº 8.019, de 1990, ...",Não informado
3,PL,755,2011,Proíbe as instituições financeiras de condicio...,Aguardando Designação de Relator
4,PL,3067,2011,"Altera o art. 9º da Lei nº 8.019, de 11 de abr...",Aguardando Parecer
